<a href="https://colab.research.google.com/github/OmriMan/Search-Engine-Wiki/blob/master/Indexer_small_corpus_anchor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Function definitions

In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 52.4 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id, rmv_stopwords=True):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  result = []
  if rmv_stopwords:
    counter = Counter([i for i in tokens if i not in all_stopwords])
  else:
    counter = Counter([i for i in tokens])
    
  for key,val in counter.items():
    tup_to_append = (key, (id, val))
    result.append(tup_to_append)
    
  return result

In [ ]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)
  

  

In [ ]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  result = postings.map(lambda x : (x[0], len(x[1])))
  return result


In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  result_before_grouping = postings.map(lambda x : (token2bucket_id(x[0]), x))
  result_after_grouping = result_before_grouping.groupByKey()
  return result_after_grouping.map(InvertedIndex.write_a_posting_list)
  
  


In [ ]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

# Inverted Index creation - body

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
# doc_title_pairs = parquetFile.select("title", "id").rdd
# from pathlib import Path 
# import os

# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#       path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
# except:
#       path = "wikidumps/*"

parquetFile_pd = pd.read_csv("small_corpus.csv")
# print(parquetFile_pd)

parquetFile = spark.createDataFrame(parquetFile_pd)
# parquetFile.show()

# parquetFile = spark.read.parquet(path)
# print(type(parquetFile))

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType

doc_text_pairs = parquetFile.select("text", "id").rdd
# doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_text_pairs = parquetFile.select("anchor_text", "id")


anchor_list = []
for i in doc_anchor_text_pairs.collect():
  anchor_as_string=''
  # print(i.anchor_text) #You can see here that the anchor text is a long string where every title is between 2 commas like this 'xxxxxx' 
  anchor_as_list = i.anchor_text.split("'")
  anchor_as_list = [anchor_as_list[i] for i in range(1,len(anchor_as_list),2)] #So after spliting between commas we save only the titles which are in all the odd places (1,3,5...)
  anchor_as_string = ' '.join(anchor_as_list) #Then transform the list to a long string with spaces between
  anchor_list.append((anchor_as_string,i.id))
  
doc_anchor_text_pairs = spark.createDataFrame(data=anchor_list, schema = ["anchor_text","id"])
doc_anchor_text_pairs=doc_anchor_text_pairs.rdd


#Code for normal corpus
# anchor_list=[]
# for i in doc_anchor_text_pairs.collect():
#   anchor_as_string=''
#   for j in i.anchor_text:
#     anchor_as_string = anchor_as_string +' ' + j[1]
#   anchor_list.append((anchor_as_string,i.id))

# doc_anchor_text_pairs = spark.createDataFrame(data=anchor_list, schema = ["anchor_text","id"])
# doc_anchor_text_pairs=doc_anchor_text_pairs.rdd

  



In [ ]:
# word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
# word_counts_title = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1], False))
word_counts_anchor = doc_anchor_text_pairs.flatMap(lambda x: word_count(x[0], x[1], False))

In [ ]:
# postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
posting_anchor_text = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)

In [ ]:
# postings_filtered_text = postings_text.filter(lambda x: len(x[1])>10)
# postings_filtered_title = postings_title.filter(lambda x: len(x[1])>10)
postings_filtered_anchor_text = posting_anchor_text.filter(lambda x: len(x[1])>10)

# text_df = calculate_df(postings_filtered_text)
# title_df = calculate_df(postings_filtered_title)
anchor_text_df = calculate_df(postings_filtered_anchor_text)

# w2df_dict_text = text_df.collectAsMap()
# w2df_dict_title = title_df.collectAsMap()
w2df_dict_anchor_text = anchor_text_df.collectAsMap()


In [ ]:
# posting_locs_list_text = partition_postings_and_write(postings_filtered_text).collect()
# posting_locs_list_title = partition_postings_and_write(postings_filtered_title).collect()
posting_locs_list_anchor_text = partition_postings_and_write(postings_filtered_anchor_text).collect()

In [ ]:
# merge the posting locations into a single dict and run more tests (5 points)
# super_posting_locs_text = defaultdict(list)
# for posting_loc in posting_locs_list_text:
#   for k, v in posting_loc.items():
#     super_posting_locs_text[k].extend(v)

# super_posting_locs_title = defaultdict(list)
# for posting_loc in posting_locs_list_title:
#   for k, v in posting_loc.items():
#     super_posting_locs_title[k].extend(v)

super_posting_locs_anchor_text = defaultdict(list)
for posting_loc in posting_locs_list_anchor_text:
  for k, v in posting_loc.items():
    super_posting_locs_anchor_text[k].extend(v)


In [ ]:
# # Create inverted index instance for text
# inverted_text = InvertedIndex()
# # Adding the posting locations dictionary to the inverted index
# inverted_text.posting_locs = super_posting_locs_text
# # Add the token - df dictionary to the inverted index
# inverted_text.df = w2df_dict_text
# # write the global stats out
# inverted_text.write_index('.', 'index_text')


# # Create inverted index instance for title
# inverted_title = InvertedIndex()
# # Adding the posting locations dictionary to the inverted index
# inverted_title.posting_locs = super_posting_locs_title
# # Add the token - df dictionary to the inverted index
# inverted_title.df = w2df_dict_title
# # write the global stats out
# inverted_title.write_index('.', 'index_title')

# # Create inverted index instance for anchor
inverted_anchor_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor_text.posting_locs = super_posting_locs_anchor_text
# Add the token - df dictionary to the inverted index
inverted_anchor_text.df = w2df_dict_anchor_text
# write the global stats out
inverted_anchor_text.write_index('.', 'index_anchor')

#Test

In [ ]:
## test the final index
pl = read_posting_list(inverted_text, 'clarity')
print(pl)

pl1 = read_posting_list(inverted_anchor_text, 'political')
print(pl1)

NameError: ignored